In [1]:
#basics
import importlib
import warnings
warnings.simplefilter("always") # using "default", warnings only appear once
import ipdb;
import os
import numpy as np
import pandas as pd
# %matplotlib qt
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

#modelbase
import modelbase
from modelbase.ode import Simulator
from scipy import integrate

from importlib.metadata import version
print(version("modelbase"))

#warnings.filterwarnings('ignore')
#plt.style.use("./notebook.mplstyle")

Could not find cannot import name 'dopri5' from 'assimulo.lib' (c:\Users\andre\miniconda3\envs\modelbase.1.30.0\lib\site-packages\assimulo\lib\__init__.py)
Could not find cannot import name 'rodas' from 'assimulo.lib' (c:\Users\andre\miniconda3\envs\modelbase.1.30.0\lib\site-packages\assimulo\lib\__init__.py)
Could not find cannot import name 'odassl' from 'assimulo.lib' (c:\Users\andre\miniconda3\envs\modelbase.1.30.0\lib\site-packages\assimulo\lib\__init__.py)
Could not find ODEPACK functions.
Could not find RADAR5
Could not find GLIMDA.


1.30.0


In [2]:
script_dir = globals().get("script_dir", os.getcwd()) # if script_dir exists, use it, otherwise use cwd
print(script_dir)
os.chdir(f"{script_dir}/..")
print(os.getcwd())

c:\Users\andre\OneDrive\Bildung\3_HHU\quant_Bio\6th\BA\QTB_modeling\analyses
c:\Users\andre\OneDrive\Bildung\3_HHU\quant_Bio\6th\BA\QTB_modeling


In [11]:
#ipdb.set_trace()
import models
importlib.reload(models)
from models import get_model

m = get_model()

stoichio_subset = m.get_stoichiometric_df().loc[:,["v2_to_P700FA-", "v3_to_P700FA", "v4_to_P700+FA", "v5_to_P700FA"]]
stoichio_subset.loc[(stoichio_subset != 0).any(axis=1)]


,v2_to_P700FA-,v3_to_P700FA,v4_to_P700+FA,v5_to_P700FA
Fd,0.0,-1.0,-1.0,0.0
P700+FA-,-1.0,0.0,-1.0,0.0
P700FA,0.0,1.0,0.0,1.0
P700FA-,1.0,-1.0,0.0,0.0
PC,1.0,0.0,0.0,1.0


In [4]:
y0 = {
    "PQ": 11.027139850905353,
    "PC": 1.8895071932002812,
    "Fd": 3.8690237263896705,
    "ATP": 1.620195002854852,
    "NADPH": 0.4882103700673736,
    "H": 0.0022147075094596015,
    "LHC": 0.8023074419510501,
    "Psbs": 0.9607146039898598,
    "Vx": 0.950783616933656,
    "PGA": 0.9913970817549008,
    "BPGA": 0.0005355311557548053,
    "GAP": 0.0062630116252017295,
    "DHAP": 0.13778623933075737,
    "FBP": 0.006126990841013743,
    "F6P": 0.31166103888161867,
    "G6P": 0.7168203893211117,
    "G1P": 0.041575582577936025,
    "SBP": 0.01311315151803723,
    "S7P": 0.15782894767619207,
    "E4P": 0.00732079113061801,
    "X5P": 0.022396849486562384,
    "R5P": 0.03751472214765548,
    "RUBP": 0.13153657267999222,
    "RU5P": 0.015005888732707041,
    "MDA": 5.85270097771621e-06,
    "ASC": 9.999994138785292,
    "H2O2": 3.4273920330125316e-06,
    "DHA": 8.513863740903352e-09,
    "GSH": 9.999999991725186,
    "GSSG": 4.137406632226743e-09,
    "TR_ox": 0.9,
    "E_inactive": 4.7368421052631575,
    "P700FA": 1,                  #"PSItot": 2.5, (in parameter vector of Matuszynska)
    "P700+FA-": 0.5,
    "P700FA-": 0.5,
}

In [5]:
m.get_stoichiometric_df().describe()

,v10,v11,v12,v13,v2_to_P700FA-,v3ASC,v3_to_P700FA,v4_to_P700+FA,v5_to_P700FA,v8,...,vPTOX,vPhosphoglucomutase,vRuBisCO,vSt12,vSt21,vStarch,vTPI,vdhap,vgap,vpga
count,33.000000,33.00,33.00,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,33.000000,...,33.000000,33.00,33.000000,33.000000,33.000000,33.000000,33.00,33.000000,33.000000,33.000000
mean,0.000000,0.00,0.00,-0.030303,0.030303,-0.030303,-0.030303,-0.060606,0.060606,-0.030303,...,0.030303,0.00,0.030303,-0.030303,0.030303,-0.060606,0.00,-0.030303,-0.030303,-0.030303
std,0.353553,0.25,0.25,0.304636,0.304636,0.394085,0.304636,0.242306,0.242306,0.304636,...,0.174078,0.25,0.394085,0.174078,0.174078,0.242306,0.25,0.174078,0.174078,0.174078
min,-1.000000,-1.00,-1.00,-1.000000,-1.000000,-2.000000,-1.000000,-1.000000,0.000000,-1.000000,...,0.000000,-1.00,-1.000000,-1.000000,0.000000,-1.000000,-1.00,-1.000000,-1.000000,-1.000000
25%,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
50%,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
75%,0.000000,0.00,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.00,0.000000,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000
max,1.000000,1.00,1.00,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,1.000000,...,1.000000,1.00,2.000000,0.000000,1.000000,0.000000,1.00,0.000000,0.000000,0.000000


In [6]:
[i for i in y0.keys() if i not in m.get_stoichiometric_df().index]

['ASC', 'GSH']

In [9]:
#warnings.simplefilter("error") #handle warnings as errors (used for DepreactionWarnings)
# warning now comes from line 565 in modelbase/core/ratemixin.py
# undo:
warnings.simplefilter("default")
s = Simulator(m)
s.initialise(y0)

In [ ]:
interval_length = 200
steps_per_second = 10
pfd_light = 1000
pfd_dark = 40

steps = steps_per_second * interval_length

s.update_parameter("pfd", pfd_light)

s.simulate(t_end = (i+1) * interval_length, steps = steps)

print(s.get_full_results_df().shape)

AttributeError: 'NoneType' object has no attribute 'shape'

In [13]:
dark_intervals = []
interval_length = 200
steps_per_second = 10
pfd_light = 1000
pfd_dark = 40

for i in range(0, 4):
    steps = steps_per_second * interval_length

    if i % 2 == 0: #if i is even
        s.update_parameter("pfd", pfd_dark) #do not set to zero
        dark_intervals.append([i * interval_length, (i + 1) * interval_length])
        print("dark")
    else:
        s.update_parameter("pfd", pfd_light)
        print("light")
    s.simulate(t_end = (i+1) * interval_length, steps = steps)
    print(f"simulated interval {i}")
    print(s.get_full_results_df().shape)

print(dark_intervals)


dark
simulated interval 0


AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
results_df = s.get_full_results_df()
flux_df = s.get_fluxes_df()

#flux_df
#results_df

In [ ]:
results_df

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(15, 8), sharex=True)
ax[0].plot(results_df.index, results_df["P700FA"], label="P700FA")
ax[0].plot(results_df.index, results_df["P700+FA"], label="P700+FA")
ax[0].plot(results_df.index, results_df["P700FA-"], label="P700FA-")
ax[0].plot(results_df.index, results_df["P700+FA-"], label="P700+FA-")
ax[0].plot(results_df.index, results_df["PC"], label="PC")
ax[0].plot(results_df.index, results_df["Fd"], label="Fd")
ax[0].legend()

ax[1].plot(flux_df.index, flux_df["vPS1"], label="vPS1")
ax[1].plot(flux_df.index, flux_df["vFdred"], label="vFdred")
#ax[1].plot(flux_df.index, flux_df["vPCox"], label="vPCox")
ax[1].legend()


for period in dark_intervals:
    for i, _ in enumerate(ax):
        ax[i].axvspan(period[0], period[1], facecolor='grey', alpha=0.5)

for i, _ in enumerate(ax):
    ax[i].set_xlim(599.98, 600.08)
    ax[i].grid(alpha = 0.5)
fig.show()

In [ ]:
folder = "figures/playground/old_model/PS1_states/"
#name = input("Enter the name of the file: ")
zoom = "600"
#zoom = "broadview"

name = f"pfd_{pfd_dark}_{pfd_light}__interval_{interval_length}__zoom_{zoom}"
# eg "200_interval__zoom_600__w_PC_Fd"
os.makedirs(folder, exist_ok=True)
fig.savefig(f"{folder}{name}.png", pad_inches = 0.1, bbox_inches = "tight")

In [ ]:
fig, ax = plt.subplots(2, 1, figsize=(15, 15), sharex=True)
ax[0].plot(results_df.index, results_df["ATP"], label="ATP")
ax[0].plot(results_df.index, results_df["NADPH"], label="NADPH")
ax[0].plot(results_df.index, results_df["Psbs"], label="Psbs")
ax[0].plot(results_df.index, results_df["H2O2"], label="H2O2")
ax[0].legend()

ax[1].plot(flux_df.index, flux_df["vPS2"], label="vPS2")
ax[1].plot(flux_df.index, flux_df["vCyc"], label="vCyc")
ax[1].plot(flux_df.index, flux_df["vRuBisCO"], label="vRubisco")
ax[1].plot(flux_df.index, flux_df["vMehler"], label="vMehler")
ax[1].legend()


for period in dark_intervals:
    for i, _ in enumerate(ax):
        ax[i].axvspan(period[0], period[1], facecolor='grey', alpha=0.5)

for i, _ in enumerate(ax):
    #ax[i].set_xlim(599.98, 600.08)
    ax[i].grid(alpha = 0.5)
fig.show()

In [28]:
folder = "figures/playground/old_model/general_understanding/"
#name = input("Enter the name of the file: ")
# eg "200_interval__zoom_600__w_PC_Fd"

#zoom = "600"
zoom = "broadview"

name = f"pfd_{pfd_dark}_{pfd_light}__interval_{interval_length}__zoom_{zoom}"

os.makedirs(folder, exist_ok=True)
fig.savefig(f"{folder}{name}.png", pad_inches = 0.1, bbox_inches = "tight")

In [21]:
os.getcwd()

'c:\\Users\\andre\\OneDrive\\Bildung\\3_HHU\\quant_Bio\\6th\\BA\\QTB_modelling'